In [ ]:
#====================================================================================================#
#                                                                                                    #
#                                                        ██╗   ██╗   ████████╗ █████╗ ██████╗        #
#      Competición - INAR                                ██║   ██║   ╚══██╔══╝██╔══██╗██╔══██╗       #
#                                                        ██║   ██║█████╗██║   ███████║██║  ██║       #
#      created:        29/10/2025  -  23:00:15           ██║   ██║╚════╝██║   ██╔══██║██║  ██║       #
#      last change:    05/11/2025  -  02:55:40           ╚██████╔╝      ██║   ██║  ██║██████╔╝       #
#                                                         ╚═════╝       ╚═╝   ╚═╝  ╚═╝╚═════╝        #
#                                                                                                    #
#      Ismael Hernandez Clemente                         ismael.hernandez@live.u-tad.com             #
#                                                                                                    #
#      Github:                                           https://github.com/ismaelucky342            #
#                                                                                                    #
#====================================================================================================#

# Competición Perretes y Gatos

## Iteración 5 - Transfer Learning VGG16

**Problema anterior**: CNN desde cero se estancó en 0.72 validation accuracy.

**Solución Transfer Learning**:
- VGG16 pre-entrenado en ImageNet (14M imágenes)
- Todas las capas convolucionales CONGELADAS
- Solo entrenamos 2 capas Dense (256 → 1)
- Learning rate bajo (0.0001) para fine-tuning
- 15 épocas (converge rápido)

**Objetivo**: Score >0.82 en Kaggle (3 submissions restantes).

In [ ]:
# Importo las librerías necesarias para trabajar con el modelo y los datos
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from tensorflow import data as tf_data
import keras

# Establezco una semilla aleatoria para que los resultados sean reproducibles
seed = 42
keras.utils.set_random_seed(seed)

# Rutas de datos - cambia solo el nombre del dataset si es diferente
DATASET_NAME = "u-tad-dogs-vs-cats-2025"
TRAIN_PATH = f"/kaggle/input/{DATASET_NAME}/train/train"
TEST_PATH = f"/kaggle/input/{DATASET_NAME}/test/test"
SUPP_PATH = f"/kaggle/input/{DATASET_NAME}/supplementary_data/supplementary_data"

print("Versión de Keras:", keras.__version__)
print("Rutas configuradas correctamente")

## Carga y Preparación de los Datos

Aquí cargo las imágenes de entrenamiento y las divido en conjunto de entrenamiento y validación.
He decidido usar un 80% para entrenar y un 20% para validar, que es una proporción bastante estándar.
Las imágenes las redimensiono a 256x256 píxeles para que todas tengan el mismo tamaño y el modelo
pueda procesarlas.

In [ ]:
# Tamaño de imagen - 224x224 para VGG16
image_size = (224, 224)

# Batch size
batch_size = 125

# Cargo las imágenes del directorio de entrenamiento
train_ds, val_ds = keras.utils.image_dataset_from_directory(
    TRAIN_PATH,
    validation_split=0.2,
    subset="both",
    seed=seed,
    image_size=image_size,
    batch_size=batch_size,
    labels="inferred",
    label_mode="binary",  # Cambio a binary para más estabilidad
)

print(f"Tamaño del conjunto de entrenamiento: {len(train_ds)} batches")
print(f"Tamaño del conjunto de validación: {len(val_ds)} batches")
print(f"Total imágenes de entrenamiento aproximadas: {len(train_ds) * batch_size}")
print(f"Total imágenes de validación aproximadas: {len(val_ds) * batch_size}")

## Data Augmentation

Defino transformaciones aleatorias más conservadoras para Transfer Learning.
VGG16 ya conoce características generales, así que no necesito tanto augmentation
como con un modelo desde cero.

In [ ]:
# Secuencia de transformaciones aleatorias aplicadas solo durante entrenamiento
# Más conservador para Transfer Learning
data_augmentation = keras.Sequential([
    keras.layers.RandomFlip("horizontal"),
    keras.layers.RandomRotation(0.1),  # ±36° suficiente
    keras.layers.RandomZoom(0.1),      # ±10% sin distorsión
    keras.layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
], name="data_augmentation")

print("Data Augmentation configurado (modo conservador para Transfer Learning)")

## Construcción del Modelo Transfer Learning

Uso VGG16 pre-entrenado en ImageNet como base. Es un modelo que ya aprendió características
generales de millones de imágenes, así que solo necesito ajustar las últimas capas para
que aprenda a diferenciar perros de gatos. Esto es mucho más rápido y efectivo que entrenar
desde cero.

Congelo todas las capas convolucionales de VGG16 y solo entreno una cabecera personalizada
simple: una capa densa de 256 neuronas con dropout y la salida binaria.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.applications import VGG16

# Forma de entrada: 224x224 con 3 canales RGB (requerido por VGG16)
input_shape = image_size + (3,)

# Cargo VGG16 pre-entrenado sin la parte superior (sin clasificación)
base_model = VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=input_shape,
    pooling='avg'  # Global Average Pooling
)

# CONGELO todas las capas de VGG16 - no las voy a entrenar
base_model.trainable = False

# Construyo el modelo completo
model = Sequential([
    keras.Input(shape=input_shape),
    data_augmentation,  # Augmentation solo en entrenamiento
    base_model,         # VGG16 congelado
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Salida binaria
], name='VGG16_Transfer_Learning')

print(f"Modelo Transfer Learning creado")
print(f"Capas VGG16 congeladas: {len(base_model.layers)}")
print(f"Solo entrenaremos: Dense(256) + Dropout + Dense(1)")
model.summary()

## Compilación y Entrenamiento del Modelo Transfer Learning

Configuro el entrenamiento con learning rate bajo (0.0001) porque estamos ajustando un modelo
ya entrenado, no necesitamos cambios tan grandes. Transfer Learning converge más rápido que
entrenar desde cero, así que con 15 épocas debería ser suficiente.

In [ ]:
%%time

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),  # LR bajo para Transfer Learning
    loss='binary_crossentropy',
    metrics=['accuracy', 'precision', 'recall']
)

epochs = 15  # Transfer Learning converge rápido

# ReduceLROnPlateau: reduce el learning rate si se estanca
reduce_lr = keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-7,
    verbose=1
)

print("Configuración Transfer Learning VGG16:")
print(f"Épocas: {epochs}")
print(f"Optimizer: Adam (lr=0.0001)")
print(f"Modelo base: VGG16 ImageNet (congelado)")
print(f"Entrenamos: Solo 2 capas Dense")
print("-" * 60)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
    callbacks=[reduce_lr],
    verbose=1
)

print("-" * 60)
print(f"Entrenamiento completado en {len(history.history['loss'])} épocas")
print(f"Val Accuracy final: {history.history['val_accuracy'][-1]:.4f}")
print(f"Val Precision final: {history.history['val_precision'][-1]:.4f}")
print(f"Val Recall final: {history.history['val_recall'][-1]:.4f}")

## Visualización de las Curvas de Aprendizaje

Aquí genero gráficos para ver cómo ha evolucionado el modelo durante el entrenamiento.
Es fundamental para detectar si hay overfitting (el modelo memoriza en vez de aprender) o
underfitting (el modelo no aprende lo suficiente).

Si las curvas de entrenamiento y validación se separan mucho, significa overfitting.
Si ambas se quedan estancadas en valores altos de pérdida, es underfitting.

In [ ]:
logs = pd.DataFrame(history.history)

plt.figure(figsize=(14, 4))

# Gráfico de pérdida
plt.subplot(1, 2, 1)
plt.plot(logs.loc[1:, "loss"], lw=2, label='Pérdida en entrenamiento')
plt.plot(logs.loc[1:, "val_loss"], lw=2, label='Pérdida en validación')
plt.xlabel("Época")
plt.ylabel("Pérdida")
plt.title("Evolución de la Pérdida")
plt.legend()
plt.grid(True, alpha=0.3)

# Gráfico de precisión
plt.subplot(1, 2, 2)
plt.plot(logs.loc[1:, "accuracy"], lw=2, label='Precisión en entrenamiento')
plt.plot(logs.loc[1:, "val_accuracy"], lw=2, label='Precisión en validación')
plt.xlabel("Época")
plt.ylabel("Precisión")
plt.title("Evolución de la Precisión")
plt.legend(loc='lower right')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n" + "="*60)
print("RESULTADOS FINALES:")
print("="*60)
print(f"Precisión final en entrenamiento: {logs['accuracy'].iloc[-1]:.4f}")
print(f"Precisión final en validación: {logs['val_accuracy'].iloc[-1]:.4f}")
print(f"Pérdida final en entrenamiento: {logs['loss'].iloc[-1]:.4f}")
print(f"Pérdida final en validación: {logs['val_loss'].iloc[-1]:.4f}")
print("="*60)

## Guardado del Modelo

Guardo el modelo entrenado para poder reutilizarlo más tarde sin tener que volver a entrenarlo.
Keras guarda toda la arquitectura, los pesos aprendidos y la configuración de compilación.

In [ ]:
model.save("model.keras")
print("Modelo guardado como 'model.keras'")

## Evaluación con Datos Suplementarios

Ahora evalúo el modelo con el conjunto de datos suplementarios que proporciona la competición.
Esto me da una idea más realista de cómo se comportará el modelo con datos completamente nuevos
que no ha visto nunca durante el entrenamiento ni la validación.

In [ ]:
supplementary_ds = keras.utils.image_dataset_from_directory(
    SUPP_PATH,
    image_size=image_size,
    batch_size=batch_size,
    labels="inferred",
    label_mode="binary",  # Binary consistente
)

print("Evaluando con datos suplementarios...")
print("-" * 60)

results = model.evaluate(supplementary_ds, return_dict=True, verbose=1)

print("-" * 60)
print("\nRESULTADOS EN DATOS SUPLEMENTARIOS:")
print(f"Precisión: {results['accuracy']:.4f}")
print(f"Pérdida: {results['loss']:.4f}")
print("-" * 60)

## Generación de Predicciones para el Conjunto de Test

Aquí genero las predicciones para todas las imágenes del conjunto de test.
Este es el conjunto que voy a usar para generar mi submission en la competición.
Es importante no modificar esta sección para mantener el formato correcto de submission.

In [ ]:
%%time

predictions_dict = {}

print(f"Generando predicciones para {len(os.listdir(TEST_PATH))} imágenes...")
print("Usando umbral 0.5 para clasificación binaria...")

for img in os.listdir(TEST_PATH):
    img_path = os.path.join(TEST_PATH, img)
    file_name = img_path.split('/')[-1]
    file_no_extension = file_name.split('.')[0]
    
    img_loaded = keras.utils.load_img(img_path, target_size=image_size)
    img_array = keras.utils.img_to_array(img_loaded)
    img_array = keras.ops.expand_dims(img_array, 0)
    
    # Predice probabilidad (salida sigmoid entre 0 y 1)
    prediction = model.predict(img_array, verbose=0)[0][0]
    
    # Conversión EXPLÍCITA con umbral 0.5
    label = 1 if prediction >= 0.5 else 0
    
    predictions_dict[int(file_no_extension)] = label

print(f"Predicciones generadas para {len(predictions_dict)} imágenes")

## Creación del Archivo de Submission

Finalmente, creo el archivo CSV con el formato requerido por la competición para enviar mis predicciones.
Verifico también que la distribución de clases sea razonable (no quiero que todas sean de una sola clase).

In [ ]:
submission = pd.DataFrame(predictions_dict.items(), columns=["id", "label"])
submission = submission.sort_values(by='id', ascending=True)
submission.to_csv('submission.csv', index=False)

print("="*60)
print("ARCHIVO DE SUBMISSION CREADO")
print("="*60)
print("\nDistribución de predicciones:")
print(submission["label"].value_counts())
print(f"\nClase 0 (Cat): {(submission['label'] == 0).sum()} imágenes ({100*(submission['label'] == 0).sum()/len(submission):.1f}%)")
print(f"Clase 1 (Dog): {(submission['label'] == 1).sum()} imágenes ({100*(submission['label'] == 1).sum()/len(submission):.1f}%)")
print(f"Total: {len(submission)} imágenes")
print("\nPrimeras filas:")
print(submission.head(10))
print("\nÚltimas filas:")
print(submission.tail(10))

# VERIFICACIÓN CRÍTICA
if (submission['label'] == 0).sum() == len(submission) or (submission['label'] == 1).sum() == len(submission):
    print("\n" + "!"*60)
    print("ALERTA: TODAS LAS PREDICCIONES SON DE UNA SOLA CLASE")
    print("Revisar el modelo antes de enviar a Kaggle")
    print("!"*60)
else:
    print("\n✓ Distribución de clases OK - Listo para enviar")

---

## Análisis de Resultados - Iteración 4 (Bug Fix)

### Cambios respecto a Iteración 3

**CORRECCIONES CRÍTICAS:**
1. **Label mode**: categorical → binary (más estable para 2 clases)
2. **Capa de salida**: softmax(2) → sigmoid(1)
3. **Loss function**: categorical_crossentropy → binary_crossentropy
4. **Predicciones**: np.argmax() → umbral explícito 0.5
5. **Verificación**: alertas si todas las predicciones son una sola clase

**OPTIMIZACIONES:**
1. **Optimizer**: RMSprop → Adam (convergencia más estable)
2. **Learning rate**: 0.001 → 0.0005 (más conservador)
3. **Épocas**: 25 → 20 (fijas, sin early stopping)
4. **Early stopping**: ELIMINADO (era demasiado agresivo)

### Hipótesis del Bug

El problema en Iteración 3 (score 0.58768) fue:
- Salida categorical con softmax puede dar probabilidades muy cercanas
- np.argmax() puede sesgar hacia una clase si el modelo no está bien calibrado
- Early stopping paró en época 15, posiblemente demasiado temprano

### Objetivos

- Validation accuracy: >0.82
- Supplementary accuracy: >0.75
- Distribución balanceada de predicciones (40-60% cada clase)
- Kaggle score: >0.75 (recuperación)

Documentación completa en ITERACIONES.md